In [ ]:
# load data and preprocessing 
import numpy as np
import time 
import json
from skimage import io,transform
import torch

with open("train/info.json") as f:
    info = json.loads(f.read())
labels = [int(image["label_id"]) for image in info]
names = [image["image_id"] for image in info]
names_labels_list = [[image["image_id"], image["label_id"]] for image in info]
names_labels_dict = dict(names_labels_list)

images = [transform.resize(io.imread("train/train/"+name),(32,32))for name in names]
images = np.array(images)

images_train = images[:50000]
images_train = images_train.reshape((12500,4,32,32,3))
labels = np.array(labels)
labels_train = labels[:50000]
labels_train = labels_train.reshape((12500,4))
images_train = np.transpose(images_train,axes=(0,1,4,2,3))

images_train = torch.from_numpy(images_train)
labels_train = torch.from_numpy(labels_train)

In [ ]:
# load data and preprocessing 
import numpy as np
import time 
import json
with open("train/info.json") as f:
    info = json.loads(f.read())

labels = [int(image["label_id"]) for image in info]
names = [image["image_id"] for image in info]
names_labels_list = [[image["image_id"], image["label_id"]] for image in info]
names_labels_dict = dict(names_labels_list)

In [ ]:
import numpy as np
from skimage import io,transform
images = [transform.resize(io.imread("train/train/"+name),(32,32))for name in names]
images = np.array(images)
print("images size:",images.shape)
io.imshow(images[0])

In [ ]:
images_train = images[:50000]
images_train = images_train.reshape((12500,4,32,32,3))
labels = np.array(labels)
labels_train = labels[:50000]
labels_train = labels_train.reshape((12500,4))
io.imshow(images_train[0][0])
images_train = np.transpose(images_train,axes=(0,1,4,2,3))
print(images_train.shape)
print(labels_train.shape)

In [ ]:
import torch
images_train = torch.from_numpy(images_train)
labels_train = torch.from_numpy(labels_train)
print(images_train.size())
print(labels_train.size())

In [ ]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class AlexNet(nn.Module):

    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Linear(256, 80)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
net = AlexNet()
print(net)

In [ ]:
params = list(net.parameters())
k = 0
for param in params:
    j = 1
    print(param.size())
    for i in param.size():
        j *= i
    k += j
print(k)

In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=0.001,momentum=0.9)

for epoch in range(5):
    running_loss = 0.0
    for i,inputs in enumerate(images_train,0):
        inputs = inputs.float()
        inputs_labels = labels_train[i] 
        inputs,inputs_labels = Variable(inputs),Variable(inputs_labels)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs,inputs_labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.data[0]
        if i%2000==1999:
            print("[%s,%s] loss:%.3f" %(epoch+1,i+1,running_loss/2000))
            running_loss = 0.0
print("Training completed!")

In [ ]:
from torch.autograd import Variable
correct = 0
total = 0
for i,inputs in enumerate(images_validation,0):
    inputs = inputs.float()
    inputs_labels = labels_validation[i]
    inputs = Variable(inputs)
    outputs = net(inputs)
    _,predicted = torch.max(outputs.data,1)
    total += inputs_labels.size(0)
    correct += (predicted == inputs_labels).sum()
print('Accuracy of the network on the validation images: %d %%' % (
    100 * correct / total))